# 22. lab 44 - obliczenia


## importy i tabele pomiarowe

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os

output_dir = "22-lab-44/report/img"
results_file = "22-lab-44/report/wyniki_obliczen.txt"
os.makedirs(output_dir, exist_ok=True)

def log_result(text, filepath=results_file):
    print(text)
    with open(filepath, "a", encoding="utf-8") as f:
        f.write(text + "\n")

def log_table(lines, filepath=results_file):
    for line in lines:
        print(line)
    with open(filepath, "a", encoding="utf-8") as f:
        for line in lines:
            f.write(line + "\n")
    print("\n") # odstęp dla czytelności

with open(results_file, "w", encoding="utf-8") as f:
    f.write("--- WYNIKI OBLICZEŃ LAB 44 ---\n\n")

# --- DANE POMIAROWE I STAŁE ---

f_net = 50.0 # Hz
omega = 2 * np.pi * f_net

R_ab = 36.0 # Ohm
R_bc = 16.0 # Ohm
R_cd = 33.0 # Ohm

I_L1_raw = np.array([10.0, 20.0, 29.9, 39.7, 49.7, 59.5, 69.1, 79.3, 89.3, 99.0])
U_L1 = np.array([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00])

I_L1L2_raw = np.array([3.7, 7.5, 11.2, 14.9, 18.8, 22.4, 26.1, 29.9, 33.7, 37.4])
U_L1L2 = np.array([0.50, 1.00, 1.50, 2.00, 2.51, 3.00, 3.50, 3.99, 4.50, 5.00])

I_L1L2L3_raw = np.array([1.5, 3.0, 4.7, 6.3, 7.9, 9.4, 11.1, 12.7, 14.3, 16.0])
U_L1L2L3 = np.array([0.48, 0.96, 1.50, 1.98, 2.50, 2.97, 3.48, 3.98, 4.49, 5.02])

I_L1 = I_L1_raw * 1e-3
I_L1L2 = I_L1L2_raw * 1e-3
I_L1L2L3 = I_L1L2L3_raw * 1e-3

U_C1 = np.array([0.48, 0.96, 1.49, 1.98, 2.49, 2.99, 3.51, 4.01, 4.53, 5.02])
I_C1_raw = np.array([0.48, 1.00, 1.60, 2.10, 2.70, 3.20, 3.80, 4.40, 5.00, 5.50])

U_C2 = np.array([0.50, 1.02, 1.50, 1.98, 2.47, 3.05, 3.52, 3.98, 4.52, 5.03])
I_C2_raw = np.array([1.0, 2.2, 3.3, 4.3, 5.5, 6.8, 7.8, 8.9, 10.1, 11.3])

U_C3 = np.array([0.48, 0.99, 1.52, 2.00, 2.53, 3.05, 3.48, 4.01, 4.51, 5.02])
I_C3_raw = np.array([2.4, 5.0, 7.7, 10.1, 12.8, 15.5, 17.7, 20.4, 22.4, 25.5])

I_C1 = I_C1_raw * 1e-3
I_C2 = I_C2_raw * 1e-3
I_C3 = I_C3_raw * 1e-3

log_result("Dane zaimportowane pomyślnie.")
log_result(f"Przyjęto częstość kołową omega = {omega:.2f} rad/s (dla f=50Hz)")
log_result(f"Przyjęto rezystancje: Rab={R_ab}, Rbc={R_bc}, Rcd={R_cd} Ohm\n")

---

## --- zadanie 1: cewki

In [ ]:
R_L1 = R_ab
R_L1L2 = R_ab + R_bc
R_L1L2L3 = R_ab + R_bc + R_cd

coils_data = [
    ("L1 (a-b)",        I_L1,      U_L1,      R_L1),
    ("L1+L2 (a-c)",     I_L1L2,    U_L1L2,    R_L1L2),
    ("L1+L2+L3 (a-d)", I_L1L2L3, U_L1L2L3, R_L1L2L3)
]

results_systems = []

for name, I, U, R in coils_data:
    res = linregress(I, U)
    a = res.slope       # = |Z|
    u_a = res.stderr    # = u(|Z|)
    b = res.intercept
    r2 = res.rvalue**2

    if a > R:
        L_sys = np.sqrt(a**2 - R**2) / omega
        dL_da = a / (omega**2 * L_sys)
        u_L_sys = dL_da * u_a
    else:
        L_sys = 0.0
        u_L_sys = 0.0

    results_systems.append({
        "name": name, 
        "L_sys": L_sys, 
        "u_L_sys": u_L_sys,
        "a": a, "u_a": u_a, "b": b, "R2": r2, "R_wire": R
    })


L_val = []
u_L_val = []

l1 = results_systems[0]['L_sys']
u_l1 = results_systems[0]['u_L_sys']
L_val.append(l1)
u_L_val.append(u_l1)

l2 = results_systems[1]['L_sys'] - results_systems[0]['L_sys']
u_l2 = np.sqrt(results_systems[1]['u_L_sys']**2 + results_systems[0]['u_L_sys']**2)
L_val.append(l2)
u_L_val.append(u_l2)

l3 = results_systems[2]['L_sys'] - results_systems[1]['L_sys']
u_l3 = np.sqrt(results_systems[2]['u_L_sys']**2 + results_systems[1]['u_L_sys']**2)
L_val.append(l3)
u_L_val.append(u_l3)

results_parts = [
    {"part": "Cewka L1", "L": L_val[0], "u_L": u_L_val[0]},
    {"part": "Cewka L2", "L": L_val[1], "u_L": u_L_val[1]},
    {"part": "Cewka L3", "L": L_val[2], "u_L": u_L_val[2]},
]


lines = []
lines.append("TABELA 1: Wyniki regresji dla badanych układów (całościowych)")
lines.append(f"{'Układ':<20} | {'|Z| [Ohm]':<12} | {'L_ukladu [H]':<15} | {'R^2':<8}")
lines.append("-" * 65)
for r in results_systems:
    lines.append(f"{r['name']:<20} | {r['a']:<12.2f} | {r['L_sys']:<15.4f} | {r['R2']:<8.4f}")
lines.append("")
log_table(lines)

lines = []
lines.append("TABELA 2: Obliczone współczynniki samoindukcji poszczególnych cewek")
lines.append("(Obliczone jako różnice indukcyjności kolejnych układów)")
lines.append("-" * 60)
lines.append(f"{'Cewka':<20} | {'L [H]':<12} | {'u(L) [H]':<12}")
lines.append("-" * 60)
for r in results_parts:
    lines.append(f"{r['part']:<20} | {r['L']:<12.5f} | {r['u_L']:<12.5f}")
lines.append("-" * 60)
lines.append(f"Całkowita L (suma):  {sum(L_val):.5f} H")
lines.append(f"L zmierzona (a-d):   {results_systems[2]['L_sys']:.5f} H")
lines.append("")
log_table(lines)


R_individual = [R_ab, R_bc, R_cd]
Z_individual = [np.sqrt(r**2 + (omega * l)**2) for r, l in zip(R_individual, L_val)]

plt.figure(figsize=(9, 6))

plt.plot(I_L1 * 1000, U_L1, 'ko', markerfacecolor='none', label='L1 (pomiary)')
plt.plot(I_L1 * 1000, I_L1 * Z_individual[0], 'k--', linewidth=1, 
         label=rf'L1 model (Z={Z_individual[0]:.1f} $\Omega$)')

I_max_2 = 5.0 / Z_individual[1]
I_line_2 = np.linspace(0, I_max_2, 10)
plt.plot(I_line_2 * 1000, I_line_2 * Z_individual[1], 'b-', 
         label=rf'L2 model (Z={Z_individual[1]:.1f} $\Omega$)')

I_max_3 = 5.0 / Z_individual[2]
I_line_3 = np.linspace(0, I_max_3, 10)
plt.plot(I_line_3 * 1000, I_line_3 * Z_individual[2], 'r-', 
         label=rf'L3 model (Z={Z_individual[2]:.1f} $\Omega$)')

plt.title('Charakterystyki prądowo-napięciowe pojedynczych cewek L1, L2, L3')
plt.xlabel('Natężenie prądu I [mA]')
plt.ylabel('Napięcie U [V]')
plt.legend()
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.tight_layout()

save_path = os.path.join(output_dir, "charakterystyka_cewek.png")
plt.savefig(save_path, dpi=300)
print(f"Wykres zapisano: {save_path}")
plt.show()

## --- zadanie 2: kondensatory ---

In [ ]:
log_result("\n--- WYNIKI OBLICZEŃ ZADANIE 2: KONDENSATORY ---")

caps_data = [
    ("C1", U_C1, I_C1, 'b', 'o'),
    ("C2", U_C2, I_C2, 'orange', 's'),
    ("C3", U_C3, I_C3, 'g', '^')
]

lines = []
lines.append("TABELA 3: Wyniki analizy kondensatorów (parametry regresji i niepewności)")
lines.append(f"{'Typ':<5} | {'a [A/V]':<12} | {'u(a) [A/V]':<12} | {'Xc [Ohm]':<10} | {'C [uF]':<10} | {'u(C) [uF]':<10} | {'R^2':<8}")
lines.append("-" * 95)

results_caps = {} 

plt.figure(figsize=(9, 6))
U_range = np.linspace(0, 5.2, 10)

for name, U, I, color, marker in caps_data:
    res = linregress(U, I)

    a = res.slope
    u_a = res.stderr
    b = res.intercept
    r2 = res.rvalue**2

    Xc = 1.0 / a

    C_val = a / omega

    u_C = u_a / omega

    results_caps[name] = C_val

    lines.append(f"{name:<5} | {a:<12.5e} | {u_a:<12.5e} | {Xc:<10.2f} | {C_val*1e6:<10.3f} | {u_C*1e6:<10.4f} | {r2:<8.4f}")

    plt.plot(U, I, marker, color=color, label=f"{name} pomiar")
    plt.plot(U_range, a*U_range + b, '--', color=color, alpha=0.7, label=rf"{name} dopasowanie")

log_table(lines)

log_result("Analiza stosunków pojemności:")
ratio_2_1 = results_caps['C2'] / results_caps['C1']
ratio_3_1 = results_caps['C3'] / results_caps['C1']
log_result(f"Stosunek C2/C1 = {ratio_2_1:.2f} (oczekiwane ~2.0)")
log_result(f"Stosunek C3/C1 = {ratio_3_1:.2f} (oczekiwane ~4.0)")

plt.xlabel("Napięcie U [V]")
plt.ylabel("Natężenie prądu I [A]")
plt.title("Charakterystyki prądowo-napięciowe kondensatorów C1, C2, C3")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

save_path = os.path.join(output_dir, "charakterystyka_kondensatorow.png")
plt.savefig(save_path, dpi=300)
print(f"Wykres zapisano: {save_path}")
plt.show()

In [ ]:
log_result("\n--- WYNIKI OBLICZEŃ ZADANIE 3: OBWÓD RLC ---")

I_RLC_raw = np.array([0.8, 1.6, 2.4, 3.3, 4.1, 4.9, 5.7, 6.6, 7.5, 8.3]) # mA
U_RLC = np.array([0.51, 0.98, 1.49, 2.02, 2.49, 2.98, 3.47, 4.00, 4.53, 5.02]) # V

I_RLC = I_RLC_raw * 1e-3 # Konwersja na A

res_rlc = linregress(U_RLC, I_RLC)
a_rlc = res_rlc.slope       # a = 1/Z [S]
u_a_rlc = res_rlc.stderr    # niepewność a
b_rlc = res_rlc.intercept
r2_rlc = res_rlc.rvalue**2

Z_meas = 1.0 / a_rlc

u_Z_meas = (u_a_rlc / a_rlc) * Z_meas

log_result(f"Wyniki regresji RLC (I = aU + b):")
log_result(f"a (1/Z) = {a_rlc:.5e} +/- {u_a_rlc:.5e} S")
log_result(f"R^2 = {r2_rlc:.4f}")
log_result(f"Zawada wyznaczona z wykresu: Z_meas = {Z_meas:.2f} +/- {u_Z_meas:.2f} Ohm")

R_tot = R_ab + R_bc + R_cd

L_tot = results_systems[2]['L_sys'] 

C_1 = results_caps['C1']

X_L = omega * L_tot
X_C = 1.0 / (omega * C_1)

Z_theo = np.sqrt(R_tot**2 + (X_L - X_C)**2)

log_result("\nObliczenia teoretyczne:")
log_result(f"R_tot = {R_tot:.1f} Ohm")
log_result(f"L_tot = {L_tot:.4f} H  =>  XL = {X_L:.2f} Ohm")
log_result(f"C_1   = {C_1*1e6:.4f} uF =>  XC = {X_C:.2f} Ohm")
log_result(f"Zawada obliczona ze wzoru:   Z_theo = {Z_theo:.2f} Ohm")

diff = abs(Z_meas - Z_theo)
log_result(f"\nRóżnica |Z_meas - Z_theo| = {diff:.2f} Ohm")
if diff < 2 * u_Z_meas:
    log_result("Wniosek: Wartość zmierzona jest zgodna z teoretyczną (w granicach 2*u(Z)).")
else:
    log_result("Wniosek: Występuje rozbieżność między modelem a pomiarem (większa niż 2*u(Z)).")

plt.figure(figsize=(9, 6))

plt.plot(U_RLC, I_RLC, 'mo', label='RLC pomiar')

U_range_rlc = np.linspace(0, 5.2, 10)
plt.plot(U_range_rlc, a_rlc * U_range_rlc + b_rlc, 'm--', alpha=0.7, 
         label=rf'Fit: $1/Z \approx {a_rlc:.4f}$ S')

plt.xlabel("Napięcie U [V]")
plt.ylabel("Natężenie prądu I [A]")
plt.title("Charakterystyka prądowo-napięciowa obwodu RLC")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

save_path_rlc = os.path.join(output_dir, "charakterystyka_RLC.png")
plt.savefig(save_path_rlc, dpi=300)
print(f"Wykres zapisano: {save_path_rlc}")
plt.show()